经过数据采集得到的数据,通常情况下是无法直接拿来用的,因为可能存在缺失值、空行、重复行等等诸多问题。因此需要对采集得到的数据做一些必要的处理，也就是数据清洗。就好比炒菜一样，大部分时间都花在准备上，如买菜、择菜、洗菜等。在整个数据分析过程中，不论是 在时间还是 功夫上，数据清洗大概占到了80%。

那么，当接到一份“新鲜”数据后，该如何着手进行数据清洗呢？有没有什么流程、方法之类的知识可以让人快速上手数据清洗呢？又如何才能得到干净、标准、连续的数据呢？这里介绍一种叫做“完全合一”的四项原则。按照“完全合一”原则进行操作，可以帮助我们解决数据清理中遇到的大部分问题。

# 数据清洗“完全合一”

- <font color=#DC143C>完</font>整性

    单条数据是否存在空值，统计的字段是否完善    

- <font color=#DC143C>全</font>面性

    观察某一列的全部数值，比如在Excel表中，选中一列，查看该列的平均值、最大值、最小值，然后通过常识来判断该列是否有问题，比如：数据定义、单位标识、数据本身。

- <font color=#DC143C>合</font>法性

    数据的类型、内容、大小的合法性。比如数据中是否存在非ASCII字符、性别存在未知、年龄大于200岁等。

- 唯<font color=#DC143C>一</font>性

    数据是否存在重复记录，因为数据通常来自不同渠道的汇总，出现重复情况在所难免。行数据、列数据都需要是唯一的，比如一个人不能重复记录多次，且一个人的体重也不能在列指标中重复记录多次。

# 利用Pandas进行数据清洗

首先导入练习数据.

In [141]:
import pandas as pd
from pandas import Series, DataFrame
path = './data/data7_example.csv'
df1 = DataFrame(pd.read_csv(path))
print(df1)

      1     micky mouse    56       70kgs   72   69   71    -  -.1  -.2
0   2.0     donald duck  34.0   154.89lbs    -    -    -   85   84   76
1   3.0      mini mouse  16.0         NaN    -    -    -   65   69   72
2   4.0  scrooge mcduck   NaN       78kgs   78   79   72    -    -    -
3   5.0    pink panther  54.0  198.658lbs    -    -    -   69  NaN   75
4   6.0     huey mcduck  52.0      189lbs    -    -    -   68   75   72
5   7.0    dewey mcduck  19.0       56kgs    -    -    -   71   78   75
6   8.0      scoopy doo  32.0       78kgs   78   76   75    -    -    -
7   NaN             NaN   NaN         NaN  NaN  NaN  NaN  NaN  NaN  NaN
8   9.0     huey mcduck  52.0      189lbs    -    -    -   68   75   72
9  10.0    louie mcduck  12.0       45kgs    -    -    -   92   95   87


经过观察,可以明显地发现,这份数据存在有缺失值、有空行、没有字段名、数据标识（单位）不统一、存在重复数据这些问题。下面按照“完全合一”的原则对这份数据进行清洗。

## 1.完整性

在实际业务中，往往因为数据量较大，会出现有些数据没有采集到的情况。通常我们可以采用以下三个方法进行处理：

- 删除：删除数据缺失的记录

- 均值：使用当前列的均值

- 高频：使用当前列出现频率最高的数据

### 问题1:缺失值

In [142]:
# 补全列名
df2 = DataFrame(
    pd.read_csv(
        path,
        names=[
            'Order', 'Name', 'Age', 'Weight', 'm0006', 'm0612', 'm1218',
            'f0006', 'f0612', 'f1218'
        ]))
df2.dropna(how='all', inplace=True)

In [143]:
# 对Age列缺失值进行填充,这里使用了均值
df2['Age'].fillna(df2['Age'].mean(), inplace=True)

### 问题2:空行

In [144]:
# 删除空行并打印
df2.dropna(how='all', inplace=True)
print(df2)

    Order            Name        Age      Weight m0006 m0612 m1218 f0006  \
0     1.0     micky mouse  56.000000       70kgs    72    69    71     -   
1     2.0     donald duck  34.000000   154.89lbs     -     -     -    85   
2     3.0      mini mouse  16.000000         NaN     -     -     -    65   
3     4.0  scrooge mcduck  36.333333       78kgs    78    79    72     -   
4     5.0    pink panther  54.000000  198.658lbs     -     -     -    69   
5     6.0     huey mcduck  52.000000      189lbs     -     -     -    68   
6     7.0    dewey mcduck  19.000000       56kgs     -     -     -    71   
7     8.0      scoopy doo  32.000000       78kgs    78    76    75     -   
9     9.0     huey mcduck  52.000000      189lbs     -     -     -    68   
10   10.0    louie mcduck  12.000000       45kgs     -     -     -    92   

   f0612 f1218  
0      -     -  
1     84    76  
2     69    72  
3      -     -  
4    NaN    75  
5     75    72  
6     78    75  
7      -     -  
9     75  

## 2.全面性

### 问题:列数据的单位不统一

Weight列的单位不统一,需要修正,这里单位统一为千克(kgs)

In [145]:
# 获取weight列中单位为lbs的数据
rows_with_lbs = df2['Weight'].str.contains('lbs').fillna(False)
print(df2[rows_with_lbs])

   Order          Name   Age      Weight m0006 m0612 m1218 f0006 f0612 f1218
1    2.0   donald duck  34.0   154.89lbs     -     -     -    85    84    76
4    5.0  pink panther  54.0  198.658lbs     -     -     -    69   NaN    75
5    6.0   huey mcduck  52.0      189lbs     -     -     -    68    75    72
9    9.0   huey mcduck  52.0      189lbs     -     -     -    68    75    72


In [131]:
# 将lbs转换为kgs,2.2lbs=1kgs
for i, lbs_row in df2[rows_with_lbs].iterrows():
    # 截取从头开始至倒数第三个字符之前的数据,也就是去掉最后的lbs
    weight = int(float(lbs_row['Weight'][:-3]) / 2.2)
    df2.at[i, 'Weight'] = '{}kgs'.format(weight)

print(df2)

    Order            Name        Age Weight m0006 m0612 m1218 f0006 f0612  \
0     1.0     micky mouse  56.000000  70kgs    72    69    71     -     -   
1     2.0     donald duck  34.000000  70kgs     -     -     -    85    84   
2     3.0      mini mouse  16.000000    NaN     -     -     -    65    69   
3     4.0  scrooge mcduck  36.333333  78kgs    78    79    72     -     -   
4     5.0    pink panther  54.000000  90kgs     -     -     -    69   NaN   
5     6.0     huey mcduck  52.000000  85kgs     -     -     -    68    75   
6     7.0    dewey mcduck  19.000000  56kgs     -     -     -    71    78   
7     8.0      scoopy doo  32.000000  78kgs    78    76    75     -     -   
9     9.0     huey mcduck  52.000000  85kgs     -     -     -    68    75   
10   10.0    louie mcduck  12.000000  45kgs     -     -     -    92    95   

   f1218  
0      -  
1     76  
2     72  
3      -  
4     75  
5     72  
6     75  
7      -  
9     72  
10    87  


## 3.合理性

### 问题:非ASCII字符

In [132]:
# # 删除非 ASCII 字符
# df['first_name'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
# df['last_name'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

## 4.唯一性

### 问题1:一列有多个参数

In [135]:
# 拆分Name列,删除原数据列
# df2.insert(1, 'FristName', df2['Name'].map(lambda x: x.split(' ')[0]))
# df2.insert(2, 'LastName', df2['Name'].map(lambda x: x.split(' ')[1]))
# del df2['Name']
# print(df2)
df2[['first_name','last_name']]=df2['Name'].str.split(expand=True)
df2.drop('Name',axis=1,inplace=True)
print(df2)

    Order        Age Weight m0006 m0612 m1218 f0006 f0612 f1218 first_name  \
0     1.0  56.000000  70kgs    72    69    71     -     -     -      micky   
1     2.0  34.000000  70kgs     -     -     -    85    84    76     donald   
2     3.0  16.000000    NaN     -     -     -    65    69    72       mini   
3     4.0  36.333333  78kgs    78    79    72     -     -     -    scrooge   
4     5.0  54.000000  90kgs     -     -     -    69   NaN    75       pink   
5     6.0  52.000000  85kgs     -     -     -    68    75    72       huey   
6     7.0  19.000000  56kgs     -     -     -    71    78    75      dewey   
7     8.0  32.000000  78kgs    78    76    75     -     -     -     scoopy   
9     9.0  52.000000  85kgs     -     -     -    68    75    72       huey   
10   10.0  12.000000  45kgs     -     -     -    92    95    87      louie   

   last_name  
0      mouse  
1       duck  
2      mouse  
3     mcduck  
4    panther  
5     mcduck  
6     mcduck  
7        doo  
9     

### 问题2:重复数据

去重可以使用drop_duplicates()来删除

In [138]:
# 删除重复数据行
df2.drop_duplicates(['first_name','last_name'], inplace=True)
print(df2)

    Order        Age Weight m0006 m0612 m1218 f0006 f0612 f1218 first_name  \
0     1.0  56.000000  70kgs    72    69    71     -     -     -      micky   
1     2.0  34.000000  70kgs     -     -     -    85    84    76     donald   
2     3.0  16.000000    NaN     -     -     -    65    69    72       mini   
3     4.0  36.333333  78kgs    78    79    72     -     -     -    scrooge   
4     5.0  54.000000  90kgs     -     -     -    69   NaN    75       pink   
5     6.0  52.000000  85kgs     -     -     -    68    75    72       huey   
6     7.0  19.000000  56kgs     -     -     -    71    78    75      dewey   
7     8.0  32.000000  78kgs    78    76    75     -     -     -     scoopy   
10   10.0  12.000000  45kgs     -     -     -    92    95    87      louie   

   last_name  
0      mouse  
1       duck  
2      mouse  
3     mcduck  
4    panther  
5     mcduck  
6     mcduck  
7        doo  
10    mcduck  


# 养成数据审核的习惯

没有高质量的数据，就没有高质量的数据挖掘，而数据清洗是高质量数据的一道保障.

![img](./img/数据清洗.jpg)